# Remote Capture

### listen

In [5]:
import pandas as pd
import scapy.all as scapy

# PCAP file path
pcap_file = './capture.pcap'

# Load the PCAP file into a list of packets
packets = scapy.rdpcap(pcap_file)

# Create a list of dictionaries to store packet information
data = []
for packet in packets:
    packet_info = {}
    if scapy.IP in packet:
        packet_info['src'] = packet[scapy.IP].src
        packet_info['dst'] = packet[scapy.IP].dst
        packet_info['protocol'] = packet[scapy.IP].proto
        packet_info['length'] = packet[scapy.IP].len
        data.append(packet_info)

# Convert the list of dictionaries to a Pandas df
df = pd.df(data)

# Print the df
df.head()

,src,dst,protocol,length
0,172.104.184.124,103.122.5.75,6,465
1,103.122.5.75,172.104.184.124,6,242
2,172.104.184.124,103.122.5.75,6,52
3,172.104.184.124,51.159.22.177,6,190
4,51.159.22.177,172.104.184.124,6,52


### extract

In [29]:
import dpkt
import socket

# Open the PCAP file
with open("capture.pcap", "rb") as f:
    pcap = dpkt.pcap.Reader(f)

    # Iterate through each packet in the PCAP file
    for ts, buf in pcap:
        # Parse the packet
        eth = dpkt.ethernet.Ethernet(buf)

        # Check if the packet is an IP packet
        if isinstance(eth.data, dpkt.ip.IP):
            ip = eth.data

            # Check if the packet is a TCP packet
            if isinstance(ip.data, dpkt.tcp.TCP):
                tcp = ip.data

                # Check if the packet contains HTTPS data
                if len(tcp.data) > 0:
                    try:
                        # Attempt to parse the HTTPS data
                        HTTPS = dpkt.HTTPS.Request(tcp.data)

                        # Print the source IP, destination IP, and HTTPS payload
                        print("Source IP: {}".format(socket.inet_ntoa(ip.src)))
                        print("Destination IP: {}".format(socket.inet_ntoa(ip.dst)))
                        print("HTTPS Payload:\n{}".format(HTTPS.body))
                    except (dpkt.dpkt.NeedData, dpkt.dpkt.UnpackError):
                        # Ignore packets that can't be parsed as HTTPS
                        pass

Source IP: 172.104.184.124
Destination IP: 103.23.22.101
HTTP Payload:
b''
Source IP: 172.104.184.124
Destination IP: 103.23.22.101
HTTP Payload:
b''
Source IP: 172.104.184.124
Destination IP: 103.23.22.101
HTTP Payload:
b''
Source IP: 172.104.184.124
Destination IP: 103.23.22.101
HTTP Payload:
b''
Source IP: 172.104.184.124
Destination IP: 103.23.22.101
HTTP Payload:
b''


In [69]:
from pcapkit import HTTP, extract
# set strict to make sure full reassembly
extraction = extract(fin='capture.pcap', store=False, nofile=True, tcp=True, strict=True)
# print extracted packet if HTTP in reassembled payloads
for datagram in extraction.reassembly.tcp:
    if datagram.packet is not None and HTTP in datagram.packet:
        data = datagram.packet
        print(datagram.packet[HTTP])
        break

47 45 54 20 2f 61 70 69 2f 6d 61 73 74 65 72 2f 6b 65 6c     GET./api/master/kel
61 73 3f 73 74 61 72 74 3d 30 26 6c 69 6d 69 74 3d 31 30     as?start=0&limit=10
30 26 6e 61 6d 61 3d 26 62 79 3d 26 61 73 63 3d 20 48 54     0&nama=&by=&asc=.HT
54 50 2f 31 2e 31 0d 0a 48 6f 73 74 3a 20 73 69 6d 70 6f     TP/1.1..Host:.simpo
6e 69 2e 34 76 69 73 69 6f 6e 6d 65 64 69 61 2e 6e 65 74     ni.4visionmedia.net
0d 0a 55 73 65 72 2d 41 67 65 6e 74 3a 20 70 79 74 68 6f     ..User-Agent:.pytho
6e 2d 72 65 71 75 65 73 74 73 2f 32 2e 32 35 2e 31 0d 0a     n-requests/2.25.1..
41 63 63 65 70 74 2d 45 6e 63 6f 64 69 6e 67 3a 20 67 7a     Accept-Encoding:.gz
69 70 2c 20 64 65 66 6c 61 74 65 0d 0a 41 63 63 65 70 74     ip,.deflate..Accept
3a 20 2a 2f 2a 0d 0a 43 6f 6e 6e 65 63 74 69 6f 6e 3a 20     :.*/*..Connection:.
6b 65 65 70 2d 61 6c 69 76 65 0d 0a 0d 0a                    keep-alive....


In [70]:
type(data)

pcapkit.protocols.application.http.HTTP

In [71]:
data

<HTTP alias='HTTP/1.1' http=(receipt=..., header=OrderedMultiDict([('Host', 'simponi.4visionmedia.net'), ('User-Agent', 'python-requests/2.25.1'), ('Accept-Encoding', 'gzip, deflate'), ('Accept', '*/*'), ('Connection', 'keep-alive')]), body=None)>

In [72]:
data.packet

Packet(header=b'GET /api/master/kelas?start=0&limit=100&nama=&by=&asc= HTTP/1.1\r\nHost: simponi.4visionmedia.net\r\nUser-Agent: python-requests/2.25.1\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n', payload=b'')

In [73]:
str(data.packet).replace("\\r", '').split("\\n")

["Packet(header=b'GET /api/master/kelas?start=0&limit=100&nama=&by=&asc= HTTP/1.1",
 'Host: simponi.4visionmedia.net',
 'User-Agent: python-requests/2.25.1',
 'Accept-Encoding: gzip, deflate',
 'Accept: */*',
 'Connection: keep-alive',
 '',
 "', payload=b'')"]

## v

In [1]:
import paramiko
import time
import os

# Function to capture packets using Scapy
def capture_packets():
    # Use Scapy to capture and save packets to PCAP file
    sniff(offline="packets.pcap")

# SSH credentials for the remote server
hostname = "172.104.184.124"
username = "root"
password = "Hehe100x"

port_listen = 443

# PCAP file path on local machine
pcap_file = './capture.pcap'

# Start the SSH connection
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(hostname, username=username, password=password)
print(f"INFO: \t Connected to Remote Server {hostname}")

# Start packet capture on remote server
cmd = 'tcpdump -s 0 -U -w - '
filter = "not port 22 and not host 51.159.22.177 and not host 51.159.53.209 and not dst host 172.104.184.124 and not dst host 103.122.5.75 and not udp"
stdin, stdout, stderr = ssh.exec_command( cmd+filter )
print(f"INFO:  \t Listening {hostname}")

# Save PCAP file on local machine
with open(pcap_file, 'wb') as f:
    while True:
        data = stdout.read(1024)
        #print( data.strip() )
        if len(data) == 0:
            break
        f.write(data)
        
# Close the SSH connection
ssh.close()

INFO: 	 Connected to Remote Server 172.104.184.124
INFO:  	 Listening 172.104.184.124


In [2]:
import json
  
# Read JSON file
f = open('out.json')
  
# returns JSON object as a dictionary
data = json.load(f)

data['Frame 57']['ethernet']['ipv4']['tcp']['http']


JSONDecodeError: Extra data: line 24 column 2 (char 384)

In [3]:
import pandas as pd

df = []
for k, v in data.items():
    try:
        receipt = v['ethernet']['ipv4']['tcp']['http']['receipt']
        header  = v['ethernet']['ipv4']['tcp']['http']['header']
        body    = v['ethernet']['ipv4']['tcp']['http']['body']

        df.append(  receipt |     \
                    header  |     \
                    {"body": body}
                )
    except:
        pass

df = pd.df(df)
df

,type,method,uri,version,Host,User-Agent,Accept-Encoding,Accept,Connection,body
0,request,GET,/api/master/kelas?start=0&limit=100&nama=&by=&...,1.1,simponi.4visionmedia.net,python-requests/2.25.1,"gzip, deflate",*/*,keep-alive,None
1,request,GET,/api/kelompok-tani/count,1.1,simponi.4visionmedia.net,python-requests/2.25.1,"gzip, deflate",*/*,keep-alive,None
2,request,GET,/api/master/komoditi?start=0&limit=100&nama=&b...,1.1,simponi.4visionmedia.net,python-requests/2.25.1,"gzip, deflate",*/*,keep-alive,None
3,request,GET,/api/kelompok-tani/perkebunan,1.1,simponi.4visionmedia.net,python-requests/2.25.1,"gzip, deflate",*/*,keep-alive,None
4,request,GET,/api/kelompok-tani/count,1.1,simponi.4visionmedia.net,python-requests/2.25.1,"gzip, deflate",*/*,keep-alive,None
5,request,GET,/api/master/komoditi?start=0&limit=100&nama=&b...,1.1,simponi.4visionmedia.net,python-requests/2.25.1,"gzip, deflate",*/*,keep-alive,None
6,request,GET,/api/master/kelas?start=0&limit=100&nama=&by=&...,1.1,simponi.4visionmedia.net,python-requests/2.25.1,"gzip, deflate",*/*,keep-alive,None
7,request,GET,/api/kelompok-tani/count,1.1,simponi.4visionmedia.net,python-requests/2.25.1,"gzip, deflate",*/*,keep-alive,None
8,request,GET,/api/kelompok-tani/perkebunan,1.1,simponi.4visionmedia.net,python-requests/2.25.1,"gzip, deflate",*/*,keep-alive,None
9,request,GET,/api/kelompok-tani/count,1.1,simponi.4visionmedia.net,python-requests/2.25.1,"gzip, deflate",*/*,keep-alive,None


## extract

In [1]:
import pandas as pd
import json
import os
import subprocess
import time
import tqdm

while True:
    print(f"INFO: \t pcapkit")
    cmd = "pcapkit-cli capture.pcap --output out.json --verbose --format json"
    try:
        os.system(cmd)
        #subprocess.run( cmd.split(" ") )
    except:
        pass
    
    print(f"INFO: \t open json")
    # Read JSON file
    f = open('out.json')
    
    # returns JSON object as a dictionary
    data = json.load(f)

    print(f"INFO: \t extract to df")
    # pandas df
    df = []
    for k, v in tqdm(data.items()):
        try:
            receipt = v['ethernet']['ipv4']['tcp']['http']['receipt']
            header  = v['ethernet']['ipv4']['tcp']['http']['header']
            body    = v['ethernet']['ipv4']['tcp']['http']['body']

            df.append(  receipt |     \
                        header  |     \
                        {"body": body}
                    )
        except:
            pass

    df = pd.df(df)
    print(df)

    time.sleep(10)
    os.system('cls')
    #subprocess.run("cls")

INFO: 	 pcapkit
INFO: 	 open json
INFO: 	 extratc to df
INFO: 	 pcapkit
INFO: 	 open json
INFO: 	 extratc to df
INFO: 	 pcapkit
INFO: 	 open json
INFO: 	 extratc to df
INFO: 	 pcapkit
INFO: 	 open json
INFO: 	 extratc to df


KeyboardInterrupt: 

# Playground

In [17]:
from flightsql import FlightSQLClient

query = """SELECT *
FROM 'census'
WHERE time >= now() - interval '24 hours'
"""

# Define the query client
query_client = FlightSQLClient(
  host = "us-east-1-1.aws.cloud2.influxdata.com",
  token = os.environ.get("INFLUXDB_TOKEN"),
  metadata={"bucket-name": "host-test"})

# Execute the query
info = query_client.execute(query)
reader = query_client.do_get(info.endpoints[0].ticket)

# Convert to dataframe
data = reader.read_all()
df = data.to_pandas().sort_values(by="time")
print(df)


ModuleNotFoundError: No module named 'flightsql'